# 课程项目：Kaggle-Titanic 泰坦尼克号获救问题

> by 马川 燕山大学

![](images/Titanic.jpg)

> 问题设置参考自网络，结合本课程进行了修改和扩充，以期为学生提供一个完整的项目案例。藉练手，贯所学，一窥数据分析之面貌。

>**代码胜于雄辩(Talk is cheap.Show me the code)** 

>               ---- 林纳斯·托瓦兹(Linus Torvalds)

> 参考链接：[**Kaggle-Titanic项目:**  https://www.kaggle.com/c/titanic](https://www.kaggle.com/c/titanic)

> 参考链接：https://blog.csdn.net/zongzi009/article/details/82391034

> 参考链接：https://github.com/massquantity/Kaggle-Titanic

> 参考链接：https://tianchi.aliyun.com/notebook-ai/detail?spm=5176.12282042.0.0.1dce2042NBc6J6&postId=6772 

本项目数据来源于[kaggle上的Titanic数据集](https://www.kaggle.com/c/titanic/data)，共计1309条数据，其中训练集891条，测试集418条。对泰坦尼克号乘客数据进行分析，**挖掘获救乘客和遇难乘客的数据特点以及与其他因素的相关性，寻找影响乘客生存与否的因素。**

**PS**:Kaggle项目还会要求根据训练集构建模型并对测试集中的数据进行预测，但本课程不涉及预测问题。

## 1. 问题及思路

### 提出问题

**1. 整体来看，存活比例如何？**

> 要求：
>    ① 将train.csv，test.csv和gender_submission.csv合并为titanic.csv
>    ② 查看已知存活数据中，存活比例如何？

**2. 结合性别和年龄数据，分析幸存下来的人是哪些人？**

> 要求：
>    ① 年龄数据的分布情况
>    ② 男性和女性存活情况
>    ③ 老人和小孩存活情况

**3. 结合 SibSp、Parch字段，研究亲人多少与存活的关系**

> 要求：
>    ① 有无兄弟姐妹/父母子女和存活与否的关系
>    ② 亲戚多少与存活与否的关系

**4. 结合票的费用情况，研究票价和存活与否的关系**

> 要求：
>    ① 票价分布和存活与否的关系
>    ② 比较研究生还者和未生还者的票价情况

**5. 称呼与存活与否的关系**

> 要求：
>    ① 观察不同称呼与生存率的关系


### 实现思路

1. 计算整体的存活率，深入分析男性和女性的不同年龄段的存活人数和占比，分析性别、年龄和存活率之间的联系；观察不同等级舱和年龄存活率，分析等级舱、年龄和存活率的关系；分析不同年龄存活率的分布来查看老人和儿童存活率。

2. 使用饼图来直观展示有父母/亲戚和没有父母/亲戚乘客的存活率有何差异。若有明显差异，可以深入分析亲人的数量对存活率是否有联系。

3. 分析所有乘客的票价分布情况，及不同等级舱的票价分布情况，通过观察幸存者和遇难者的票价的平均值或标准差来分析票价与存活率的联系；通过观察不同称呼与生存率的关系，分析社会地位对生存率的影响。

## 2. 理解数据

从Kaggle平台上下载3个原始数据集：train.csv，test.csv，gender_submission.csv。

其中：
1. train.csv 为编号为 1-891 共 891 名乘客的信息，作为训练数据；
2. test.csv 为编号 892-1309 共 418 名乘客的信息，作为测试数据(删掉了是否生还列 Survived)；
3. gender_submission.csv 为 Kaggle 项目需要提交的结果格式以及正确答案(乘客的实际生还情况)。 

### 导入相关包

In [ ]:
#导入需要的包
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

### 读取数据

数据在data文件夹里

In [ ]:
#读取数据
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
gender_submission = pd.read_csv('data/gender_submission.csv')

train存放的是训练数据，test存放的是测试数据，gender_submission 存放的是 test 中418名乘客的实际生还情况。

**相关字段含义介绍：**

* PassengerId : 乘客ID

* Survived： 是否生还

* Pclass : 乘客等级(1/2/3等舱位)

* Name : 乘客姓名

* Sex : 性别

* Age : 年龄

* SibSp : 堂兄弟/妹个数

* Parch : 父母与小孩个数

* Ticket : 船票信息

* Fare : 票价

* Cabin : 客舱

* Embarked : 登船港口

## 3. 数据清洗

数据清洗主要包括：**选取子集，缺失数据处理、数据格式转换、异常值数据处理**等。

### 基本数据处理

为了获得更完整的数据，先将 test 与 gender_submission **合并**，再将合并后的数据与 train 中的数据**连接**在一起，以得到 1309 条完整数据：

**合并**

以 PassengerId 为键值，left合并方式合并

In [ ]:
full_test = pd.merge(test,gender_submission,on='PassengerId',how='left')  #合并数据
full_test.head()

**连接**

连接后的完整数据赋值为 **titanic**

In [ ]:
titanic = pd.concat([train,full_test],ignore_index=True)
titanic.head()

**查看缺失值**

In [ ]:
titanic.info()

In [ ]:
titanic.isnull().sum()

“Age”、“Cabin”、“Embarked”、“Fare”列有缺失值。

### 缺失值处理的方法

对数据进行分析的时候要注意其中是否有缺失值。对于缺失值，一般有以下几种处理方法:

1. 如果数据集很多，但有很少的缺失值，可以删掉带缺失值的行。

2. 如果该属性相对学习来说不是很重要，可以对缺失值赋均值或者众数。

3. 对于标称属性，可以赋一个代表缺失的值，比如‘U0’。因为缺失本身也可能代表着一些隐含信息。比如船舱号Cabin这一属性，缺失可能代表并没有船舱。

比如在哪儿上船Embarked这一属性（共有三个上船地点），缺失两个值，可以用众数赋值。

PS: 众数(Mode)是指一组数据中出现次数最多的那个数据

#### 填充缺失值

In [ ]:
# 填充 Embarked 中的缺失值
titanic.Embarked[titanic.Embarked.isnull()] = titanic.Embarked.dropna().mode().values

# 填充 Fare 中的缺失值 (因为“Fare”主要与“Pclass”有关,所以我们用3等舱的中间票价来填充缺失的值)
titanic.Fare.fillna(titanic[titanic.Pclass==3]['Fare'].median(),inplace=True)

# 填充 Cabin 中的缺失值
# 缺失可能代表并没有船舱，可能有船舱号的人和没有船舱号的人的存活率有差异。因此，将有船舱号的人和没有船舱号的人进行区分。
titanic.loc[titanic.Cabin.notnull(),'Cabin'] = 1
titanic.loc[titanic.Cabin.isnull(),'Cabin'] = 0

# 填充 Age 中的缺失值
# 通过分析可知年龄和称呼关系比较密切,因此，将相同称呼的乘客用中位数来填充年龄缺失值
titanic['Title'] = titanic['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
# 映射“称呼”
nn = {  'Capt':'Rareman', 
        'Col':'Rareman',
        'Don':'Rareman',
        'Dona':'Rarewoman',
        'Dr':'Rareman',
        'Jonkheer':'Rareman',
        'Lady':'Rarewoman',
        'Major':'Rareman',
        'Master':'Master',
        'Miss':'Miss',
        'Mlle':'Rarewoman',
        'Mme':'Rarewoman',
        'Mr':'Mr',
        'Mrs':'Mrs',
        'Ms':'Rarewoman',
        'Rev':'Mr',
        'Sir':'Rareman',
        'the Countess':'Rarewoman' }
titanic.Title = titanic.Title.map(nn)
# 单独处理一下 797 号乘客
titanic.loc[titanic.PassengerId == 797,'Title'] = 'Rarewoman'
# 将年龄缺失值先统一赋值为 999
titanic.Age.fillna(999,inplace=True)
# 找出小女孩
def girl(pax):
    if (pax.Age != 999) & (pax.Title == 'Miss') & (pax.Age <= 14):
        return 'Girl'
    elif (pax.Age == 999) & (pax.Title == 'Miss') & (pax.Parch != 0):
        return 'Girl'
    else:
        return pax.Title
titanic['Title'] = titanic.apply(girl,axis=1)
# 接下来，我们根据他们的“称呼”来填写缺失的“年龄”。
Tit = ['Mr','Miss','Mrs','Master','Girl','Rareman','Rarewoman']
for i in Tit:
    titanic.loc[(titanic.Age == 999) & (titanic.Title == i),'Age'] = titanic.loc[titanic.Title == i,'Age'].median()

**至此，缺失值已经全部填充完成。**

In [ ]:
titanic.info()

In [ ]:
titanic.describe()

mean字段告诉我们，大概0.377387的人最后获救了，2/3等舱的人数比1等舱要多，平均乘客年龄大概是30.3岁

## 4. 数据分析及可视化

在进行数据分析及可视化时，**依据每个问题选取所需切片并赋值给新的 DataFrame 再进行分析和可视化展示，这样数据不易乱且条理更清晰。**

![](images/news.jpg)

**正常显示中文的设置**

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei'] #用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False #用来正常显示负号

# plt.style.use('ggplot')   # R 中的 ggplot 的效果

### （1）整体来看，存活比例如何？

获取生还比例 (即统计 Survived 列中值 0 和 1 的出现次数)

In [ ]:
titanic['Survived'].value_counts()

可视化生还**占比**情况，试试**饼图**吧。

### （2）结合性别和年龄数据，分析幸存下来的人是哪些人？

>要求：
>① 年龄数据的分布情况
>② 男性和女性存活情况
>③ 老人和小孩存活情况

**灾难面前，生死存亡之际，男士们是否让妇孺优先，展现出了高贵的骑士精神？**

![](images/titanic01.jpg)

**选取 titanic 的['Survived','Age','Sex']几列，得到一个 DataFrame ，赋值为 age_df**

In [ ]:
# 筛选年龄和性别数据
age_df = titanic.loc[:, ['Survived','Age','Sex']]
age_df.head()

试分析年龄的**分布**情况。

试试**直方图**吧

**分析男女生还比例**

提示：将 age_df 分组，分组标签为'Sex'，并求其平均值

In [ ]:
age_df.groupby(['Sex']).mean()

以上为不同性别的生存率，可见在泰坦尼克号事故中，还是体现了 **Lady First**.

试试绘制不同年龄下的平均生存率的**柱形图**或**折线图吧**

**绘制年龄与生存率的散点图**

根据分析可得以下结论： 

* 在这场事故中乘客存活率是37.74%，获救的乘客占总人数的小部分。 

* 女性存活率为83.62%,男性存活率为12.93%,大部分女性获救，绝大部分男性乘客都遇难了，遇难最多的男性是20~30岁的青年男性。 

* 男性乘客中除了0-10岁年龄段的存活率与女性相近以外，其他年龄段都远远低于女性，女性乘客中存活率最高的是60~70年龄段的。 

>说明当时现场救援是按照女士，老人和小孩优先的原则进行的。 

* 1等舱幸存者人数相比2,3等级要多,等级越高幸存者整体年龄越大，等级2,3中有较多10岁以下儿童幸存

### （3）结合 SibSp、Parch字段，研究亲人多少与存活的关系

>要求：
>① 有无兄弟姐妹/父母子女和存活与否的关系
>② 亲戚多少与存活与否的关系

**亲情激发之牵挂，是否更能生发出生之希望？**

> 父兮生我，母兮鞠我。抚我畜我，长我育我，顾我复我，出入腹我。欲报之德。昊天罔极! ----《诗经·小雅·蓼莪》

![](images/titanic02.jpg)

**选取 titanic 的['Survived','SibSp','Parch']几列，得到一个 DataFrame ，赋值为 sp_df**

In [ ]:
#筛选SibSp、Parch数据
sp_df = titanic.loc[:, ['Survived','SibSp','Parch']]
sp_df.head()

**亲友的人数和存活与否的关系 SibSp & Parch**

在一个坐标系(子图)中可以混用多种图形。比如我们可以用柱形图和折线图混用来展示 Titanic 数据集中亲人数和生存率之间的关系。

【参见 ```06 Matplotlib.ipynb``` 中的```3.7 综合运用```】

从图表中可以看出，若独自一人，那么其存活率比较低；但是如果亲友太多的话，存活率也会很低。

根据分析可得以下结论： 

* 家庭成员在1-3人的乘客的存活率相比单生一人和家庭成员人数超过3人的乘客更高。 

* 有1-2个小孩的乘客的存活率更高。 

* 说明在救援中对有小孩的乘客会优先照顾，但是小孩太多则可能会因为公平原则无法优先考虑。

### （4）结合票的费用情况，研究票价和存活与否的关系

>要求：
>① 票价分布和存活与否的关系
>② 比较研究生还者和未生还者的票价情况

![](images/titanic03.jpg)

**选取 titanic 的['Survived','Pclass','Fare']几列，得到一个 DataFrame ，赋值为 fare_df**

In [ ]:
fare_df = titanic.loc[:, ['Survived','Pclass','Fare']]

**绘制乘客票价直方图**

**分析票价和存活与否的关系**

In [ ]:
fare_df.groupby(['Pclass']).mean()
# fare_df.groupby(['Pclass']).agg({'Survived':'sum','Fare':'mean'})
# fare_df.groupby('Survived').mean()
# fare_df.groupby(['Survived','Pclass']).mean()

根据分析可得以下结论： 

* 票价整体呈现指数下降，绝大部分乘客的票价都比较低。从等级舱票价箱型图可知，不同等级舱的票价相差很大，3等舱的大部分票价都不超过20. 

* 幸存者的平均票价高于遇难者，幸存车票价票价分布差异较大，遇难者票价分布差距较小。 

* 说明在救援中票价高的人存活率更高，在救援中按等级舱分为不同的优先等级。

### （5）称呼与存活与否的关系分析

> 能力越大,责任就越大!

> With great power comes great responsibility！

>                                    ---- 蜘蛛侠

**能力越大,责任就越大! 真的是这样吗？**

![](images/titanic04.jpg)

**观察不同称呼与生存率的关系**

选取 titanic 的['Title','Age','Survived']几列，得到一个 DataFrame ，并将其分组，分组标签为['Title']，并求其平均值,命名为 title_df

In [ ]:
# 构建数据
title_df = titanic.loc[:,['Title','Age','Survived']].groupby(['Title']).mean()

**绘制不同称呼与生存率的关系的柱形图，绘制不同称呼乘客的平均年龄柱形图，最好两张图并排放置。(提示：plt.subplots( nrows=1, ncols=2))**

“贵妇人”的生存率是100%，真是太让人惊奇了！！

## 5. 总结

> 梳理分析结果形成结论。通过提炼隐藏在数据背后的信息，总结出所研究对象的内在规律的目的，以期获得有价值的产品、服务，或深刻的洞见!

* **<font color='red'>该项目是一个经典的数据分析练手项目</font>**。这里仅选取了几个主要的因素，分析其与乘客的存活率之间的联系。尚有很多可继续分析挖掘之处，如不同等级舱不同性别各个年龄段的存活率等。

* **<font color='red'>功夫在诗外</font>**。更深入的分析需要更多的数理统计方面的知识(如统计学，线性代数等)，以及对数据的理解。如此，才能熟稔数据分析背后的原理。

* **<font color='red'>聊抛一砖以引玉，盼我燕大诸子尽心之、努力之</font>。**